# Tarea 5 - Diferenciación automática - Parte 2

Autores: Ignacio Vargas Cordero, Alejandra Yuriko Yamamoto Elizalde

La extensión a estas funciones es elemental, y así por ejemplo tenemos:

\begin{eqnarray}
\exp[\vec{f}(x_0)] &=& \big( \exp(f_0),\, f'_0\exp(f_0) \,\big),\\
\log[\vec{f}(x_0)] &=& \big( \log(f_0),\, \frac{f'_0}{f_0} \,\big),\\
\sin[\vec{f}(x_0)] &=& \big( \sin(f_0),\, f'_0\cos(f_0) \,\big),\\
\cos[\vec{f}(x_0)] &=& \big( \cos(f_0),\, -f'_0\sin(f_0) \,\big),\\
\tan[\vec{f}(x_0)] &=& \big( \tan(f_0),\, f'_0\sec^2(f_0) \,\big),\\
\sinh[\vec{f}(x_0)] &=& \big( \sinh(f_0),\, f'_0\cosh(f_0) \,\big),\\
...\\
\end{eqnarray}

Vale la pena enfatizar que la regla de la cadena se utiliza en la segunda componente del 
lado derecho de estas ecuaciones.

## I

Implementen las funciones relacionadas con potencias, exponenciales, logarítmos, las funciones trigonométricas y las hiperbólicas que tiene Julia definidas (ver la [documentación](http://julia.readthedocs.org/en/release-0.4/manual/mathematical-operations/#trigonometric-and-hyperbolic-functions)). La implementación debe incluir los tests adecuados. La manera en que implementen las cosas debe estar clara en el notebook "Tarea5.ipynb", pero toda la funcionalidad debe ser incluida en "AutomDiff.jl" al igual que en "runtests.jl"

**Hint:** Aprovechen las capacidades de Julia en metaprogramming.

In [ ]:
include("AutomDiff.jl")
#include("runtest.jl")
using AD

In [ ]:
import Base: exp, log, sin, cos, tan, sinh, cosh, tanh

# Aqui se implementan los métodos necesarios para cada función

#Exponenciales
exp(a::Dual) = Dual(exp(a.fun), a.der * exp(a.fun)) 


#Logaritmos
log(a::Dual) = Dual(log(10,a.fun), a.der / a.fun) 


#Funciones Trigonométricas
sin(a::Dual) = Dual(sin(a.fun), a.der * cos(a.fun)) 

cos(a::Dual) = Dual(cos(a.fun), -a.der * sin(a.fun))

tan(a::Dual) = Dual(tan(a.fun), a.der * sec^2(a.fun))
 

#Funciones Hiperbólicas
sinh(a::Dual) = Dual(sinh(a.fun), a.der * cosh(a.fun))
 
cosh(a::Dual) = Dual(cosh(a.fun), a.der * sinh(a.fun))

tanh(a::Dual) = Dual(tanh(a.fun), a.der - a.der*tanh^2(a.fun))



## II

Prueben su implementación calculando la derivada de $f(x) = \sin\Big(x^3 - \frac{2}{x^6}\Big)$
en $x_0 = 2$.

Dibujen, para $x_0 \in [1,5]$ la función $f'(x)$.

In [ ]:
f(x) = sin(x^3 - 2/x^6)

In [ ]:
#Derivada

#Como x0 = 2, calculamos. La segunda entrada es la derivada de f(x) evaluada en x0 = 2

f(xdual(2))

In [ ]:
#Plot
using PyPlot

In [ ]:
h = linspace(1.0, 5.0, 500);
derivada = zeros(Float64, 500);
k = 1
for i in h
    derivada[k] = f(xdual(i)).der
    k = k + 1
end

In [ ]:
plot(h, derivada);
xlim(0.5, 5.0)
ylim(-60.0, 60.0)
xlabel("h")
ylabel("Derivada")

# III

Implementen el método de Newton para una función arbitraria $f(x)$ en una variable (la función `newton1D` debe tener un argumento que es una función) a fin de encontrar un cero de la función, i.e., $f(x^*)=0$. El método de Newton requiere la derivada evaluada en diversos puntos, cosa que obtendrán usando el módulo de derivación automática.

- Obtengan, usando su función del método de Newton, las raices del polinomio de Wilkinson $W_{6}(x)$, usando como iterados iniciales $x_0=2.2$ y $x_0=2.45$.
    
- Hagan lo mismo que en el inciso anterior para $g(x) = x \sin(2x)$, con $x_0=0.7$.

In [ ]:
#Raices del polinomio de Wilkinson
W6(x) = (x+1)*(x+2)*(x+3)*(x+4)*(x+5)*(x+6)


In [ ]:
#Metodo de Newton para f(x)

function newton1D(f::Function, x0::Real)
    
    xn = xdual(x0)  #Da como resultado (x0, 1)
    fxd = f(xn)
    
    for i in 0:50
        
        xn == xn - fxd.fun/fxd.der
    end
    
    return xn
end

In [ ]:
newton1D(W6, 2.2)

In [ ]:
newton1D(W6, 2.45)

In [ ]:
g(x) = x*sin(2x)


In [ ]:
newton1D(g, 0.7)

# IV

¿Qué hacer si se requieren derivadas de orden superior, es decir, la segunda, tercera, etc, derivada? Argumenten.

**NOTA:** Esta pregunta debe ser contestada pero **no** será considerada para aceptar la tarea; la idea es empezar a pensar en esto.

Se utiliza un vector con *n+1* número de entradas, donde la primer componente será la función, la segunda entrada  será la primer derivada, etc. Por lo que el vector tendrá las primeras *n* derivadas de la función dada.